In [1]:
from datetime import datetime
import csv
import pandas as pd
import pandas_profiling
import numpy as np

In [2]:
dfs = []

for x in range(ord('A'), ord('Z')+1):
    dfs.append(pd.read_csv(f'storage/data-02-option_data.{chr(x)}.csv'))

df = pd.concat(dfs)

# FILTERS
df = df[df.tradability == 'tradable']

# DROP COLUMNS
df.drop(['chain_id'], axis=1, inplace=True)
df.drop(['chain_symbol'], axis=1, inplace=True)
df.drop(['created_at'], axis=1, inplace=True)
df.drop(['high_fill_rate_buy_price'], axis=1, inplace=True)
df.drop(['high_fill_rate_sell_price'], axis=1, inplace=True)
df.drop(['id'], axis=1, inplace=True)
df.drop(['instrument'], axis=1, inplace=True)
df.drop(['instrument_id'], axis=1, inplace=True)
df.drop(['issue_date'], axis=1, inplace=True)
df.drop(['low_fill_rate_buy_price'], axis=1, inplace=True)
df.drop(['low_fill_rate_sell_price'], axis=1, inplace=True)
df.drop(['min_ticks'], axis=1, inplace=True)
df.drop(['occ_symbol'], axis=1, inplace=True)
df.drop(['rhs_tradability'], axis=1, inplace=True)
df.drop(['sellout_datetime'], axis=1, inplace=True)
df.drop(['state'], axis=1, inplace=True)
df.drop(['tradability'], axis=1, inplace=True)
df.drop(['updated_at'], axis=1, inplace=True)
df.drop(['url'], axis=1, inplace=True)

df = df[df.bid_size > 50]
df = df[df.mark_price > .30]
df = df[df.type == 'put']
df = df[df.volume > 50]
df = df[df.strike_price < 8.00]

df['percent_profit'] = df.mark_price / df.strike_price

today = datetime.now()
df.expiration_date = pd.to_datetime(df.expiration_date)
df['days_held'] = df.expiration_date.apply(lambda x: (x - today).days)
df['annualized'] = (df.percent_profit / df.days_held) * 365

df = df[df.annualized > 1.0]
df.shape

(63, 30)

In [3]:
df.head(n=40)

,expiration_date,strike_price,type,adjusted_mark_price,ask_price,ask_size,bid_price,bid_size,break_even_price,high_price,...,chance_of_profit_short,delta,gamma,implied_volatility,rho,theta,vega,percent_profit,days_held,annualized
6819,2021-09-17,5.0,put,2.00,2.05,87,1.95,174,3.00,2.00,...,0.472785,-0.430569,0.114587,1.537793,-0.011347,-0.006073,0.008760,0.400000,104,1.403846
6829,2021-06-18,5.0,put,1.38,1.40,14,1.35,112,3.62,1.46,...,0.505879,-0.590457,0.208856,1.981612,-0.002027,-0.018412,0.003650,0.275000,13,7.721154
6840,2021-12-17,7.5,put,4.35,4.50,384,4.20,838,3.15,4.40,...,0.391615,-0.529994,0.094751,1.394675,-0.035893,-0.004114,0.011976,0.580000,195,1.085641
32577,2021-06-18,7.5,put,2.13,2.20,85,2.05,88,5.37,2.15,...,0.472033,-0.881192,0.148318,1.062474,-0.003717,-0.006759,0.002504,0.283333,13,7.955128
33499,2021-06-18,7.5,put,0.60,0.65,1,0.55,239,6.90,0.60,...,0.575632,-0.609609,0.390438,0.593328,-0.002662,-0.009605,0.006358,0.080000,13,2.246154
38024,2021-07-16,7.5,put,1.00,1.05,1,0.95,265,6.50,1.05,...,0.585047,-0.493553,0.189348,0.808041,-0.005950,-0.008810,0.010399,0.133333,41,1.186992
38033,2021-06-18,7.5,put,0.73,0.75,15,0.70,180,6.77,0.80,...,0.589768,-0.537093,0.278201,0.853565,-0.002473,-0.014455,0.006644,0.096667,13,2.714103
38726,2021-06-18,2.5,put,0.38,0.40,10,0.35,831,2.12,0.40,...,0.568083,-0.525553,0.557842,1.316448,-0.000864,-0.007245,0.002158,0.150000,13,4.211538
40069,2021-06-18,7.5,put,0.50,0.80,522,0.20,247,7.00,0.45,...,0.703938,-0.277203,0.134411,1.259664,-0.001539,-0.021296,0.006628,0.066667,13,1.871795
42310,2021-06-18,5.5,put,0.37,0.40,11,0.33,223,5.13,0.42,...,0.563881,-0.666079,0.688717,0.435213,-0.002072,-0.004891,0.004417,0.066364,13,1.863287


In [6]:
# expiration_date	strike_price	adjusted_mark_price	ask_price	ask_size	bid_price	bid_size	break_even_price	high_price	last_trade_price	last_trade_size	low_price	mark_price	open_interest	previous_close_date	previous_close_price	volume	symbol	chance_of_profit_long	chance_of_profit_short	delta	gamma	implied_volatility	rho	theta	vega	percent_profit	days_held	annualized			
columns = [
    'annualized',
    'expiration_date',
    'rating',
    'symbol',
    'strike_price',
    'mark_price',
]
df['rating'] = ''
df.to_csv('data-04-interesting.csv', header=True, index=False, quoting=csv.QUOTE_NONNUMERIC, columns=columns)